<a href="https://colab.research.google.com/github/DenZhid/TelecommunicationSystemsLabs/blob/main/Lab7/Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Зависимости**

In [1]:
import numpy as np
PI2 = 2 * np.pi

# **Упражнение 7.1**

В блокноте для этой главы, *chap07.ipynb*, представлены дополнительные примеры и пояснения. Прочитайте блокнот и запустите код.

**Ход работы:**

В представленном блокноте была изучена работа дополнительных примеров и пояснения к ним.

# **Упражнение 7.2**

В этой главе показано, как выразить ДПФ и обратное ДПФ произведениями матриц. Время выполнения этих операций пропорционально *$N^2$*, где *$N$* - длина массива сигнала. Во многих случаях этого достаточно, но есть алгоритм и побыстрее - быстрое преобразованье Фурье (БПФ); время его работы пропорционально *$NlogN$*.

Ключ к БПФ - это лемма Дэниелсона-Ланцоша (Danielson-Lanczos):

$DFT()[n]=DFT(e)[n]+exp(-2i\pi n/N)DFT(o)[n]$

где $DFT(y)[n]$ - этой $n$-й элемент ДПФ от $y$, $e$ и $o$ - массивы сигнала, содержащие соответственно четные и нечетные элементы $y$ю

Эта лемма предлагает рекурсивный алгоритм для ДПФ:
1.   Дан массив сигнала $y$. Разделим его на четные и элементы $e$ и нечетные элементы $o$.
2.   Вычислим $DFT$ $e$ и $o$, делая рекурсивные вызовы.
3.   Вычислим $DFT(y)$ для каждого значения $n$, используя лемму Дэниелсона-Ланцоша.

В простешем случае эту рекурсию надо продолжать, пока длина $y$ не дойет до 1. Тогда $DFT(y)$ - $y$. А если длина $y$ достаточно мала, можно вычислить его ДПФ перемножением матриц, используя заранее вычисленные матрицы.

Подсказка: реализовывать этот алгоритм стоит постепенно, начав с нерекурсивной версии. На шаге 2, вместо того, чтобы делать рекурсивный вызов, используйте *dft*, как показано в разделе "ДПФ" на стр. 93, или *np.fit.fit*. Отладьте шаг 3 и проверьте, согласуются ли результаты с другими реализациями. Затем добавьте базовый случай и убедитесь, что он работает. И наконец, замените шаг 2 на рекурсивные вызовы.

Примечание: помните, что ДПФ периодично; попробуйте применить *np.title*.

О БПФ можно прочитать подробнее в Википедии: [https://ru.wikipedia.org/wiki/Быстрое_преобразование_Фурье](https://ru.wikipedia.org/wiki/Быстрое_преобразование_Фурье).

**Ход работы:**

Начнём с небольшого сигнала и вычсислим для него БПФ:

In [2]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)

Имплементируем алгоритм ДПФ без рекурсии (представленный в книге):

In [3]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [4]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

Данная реализация даёт такой же результат.

Теперь попробуем разбивать входной массив и использовать функцию БПФ, используемую выше для вычисления половин.

In [9]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [10]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

И снова результат совпал (правда теперь он округлился до 0).

А теперь заменим вызовы функции БПФ, рекурсивными вызовами.

In [7]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Посмотрим на результат.

In [8]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

Результат снова оказался приблежённым к первичному.